# Análisis de componentes principales

En este notebook utilizaremos el análisis de componentes principales con el fin de resumir el comportamiento de 5 índices accionarios en un sólo índice. Este nuevo índice corresponderá al primer componente principal de nuestra matriz de datos

In [4]:
#lee los datos
import pandas as pd
import numpy as np

archivos_csv = ('dow30.csv', 'naftrac.csv', 'nasdaq.csv', 'russell2000.csv', 'sp500.csv')
ruta = 'datos/'

#crea el nombre de las columnas a partir del nombre de cada csv
nombres_columnas = [s.split('.')[0] for s in archivos_csv]

#Se lee el primer archivo csv
#Solo columas Date y Adj Close
columnas = ['Date', 'Adj Close']
datos = pd.read_csv(ruta + archivos_csv[0], usecols = columnas)

#Renombra columna 'Adj Close' (in place)
datos.rename(columns = {'Adj Close': nombres_columnas[0]}, inplace = True)

#Une los datos
for i in range(1, len(archivos_csv)):
    #lee el siguiente archivo csv
    d2 = pd.read_csv(ruta + archivos_csv[i], usecols = columnas)
    
    #Hace un merge utilizando la columna 'Date' como llave
    datos = pd.merge(datos, d2, how = 'inner', on = 'Date', suffixes= ['_' + nombres_columnas[i-1], '_' + nombres_columnas[i]])
    
    #renombra columna 'Adj Close'
    datos.rename(columns = {'Adj Close': nombres_columnas[i]}, inplace = True)

#libera memoria
del d2

#Vista a los primeros 10 renglones
datos.head(10)


,Date,dow30,naftrac,nasdaq,russell2000,sp500
0,2014-08-07,16368.269531,41.218311,4334.970215,1119.760010,1909.569946
1,2014-08-08,16553.929688,41.171616,4370.899902,1131.349976,1931.589966
2,2014-08-11,16569.980469,41.732018,4401.330078,1141.930054,1936.920044
3,2014-08-12,16560.539063,41.769382,4389.250000,1133.030029,1933.750000
4,2014-08-13,16651.800781,41.769382,4434.129883,1141.780029,1946.719971
5,2014-08-14,16713.580078,41.816078,4453.000000,1143.339966,1955.180054
6,2014-08-15,16662.910156,41.666649,4464.930176,1141.650024,1955.060059
7,2014-08-18,16838.740234,41.881470,4508.310059,1158.400024,1971.739990
8,2014-08-19,16919.589844,42.002880,4527.509766,1162.469971,1981.599976
9,2014-08-20,16979.130859,42.236385,4526.479980,1157.510010,1986.510010


In [5]:
#Funcion para calcular los rendimientos
def calcula_rendimientos(datos, nombres_columnas):
    '''
    Función para calcular los rendimientos (aritméticos) de cada instrumento
    
    ENTRADA:
    datos. Pandas dataframe con los precios de cada instrumento
    
    nombres_columnas. lista con los nombres de las columnas para las que se
    quiere calcular el rendimiento
    
    SALIDA:
    pandas dataframe con los rendimientos de cada instrumento
    '''
    
    #Número de observaciones
    n_obs = datos.shape[0]
    
    #dataframe con los rendimientos
    #la primera fecha no tendrá rendimiento (se omite ya que no hay dato previo)
    rendimientos = pd.DataFrame()
    rendimientos['Date'] = datos['Date'][1:n_obs]
    
    #Reinicia los índices
    rendimientos = rendimientos.reset_index(drop = True)
    
    #Calcula el rendimiento de cada columna
    for columna in nombres_columnas:
        
        #revisa que no haya un precio en 0 o negativo
        if np.all(datos[columna] > 0):
            
            #Ojo con el reset index
            numerador = datos[columna][1:n_obs]
            numerador = numerador.reset_index(drop = True)
            denominador = datos[columna][0:(n_obs - 1)]
            denominador = denominador.reset_index(drop = True)
            
            rendimientos[columna] = numerador / denominador - 1 
        else:
            print('ERROR')
            print('Para la columna', columna,'se tiene un valor de cero o negativo en alguna fecha')
            return 0
    
    return rendimientos

In [6]:
rendimientos = calcula_rendimientos(datos, nombres_columnas)
rendimientos.head(10)

,Date,dow30,naftrac,nasdaq,russell2000,sp500
0,2014-08-08,0.011343,-0.001133,0.008288,0.010350,0.011531
1,2014-08-11,0.000970,0.013611,0.006962,0.009352,0.002759
2,2014-08-12,-0.000570,0.000895,-0.002745,-0.007794,-0.001637
3,2014-08-13,0.005511,0.000000,0.010225,0.007723,0.006707
4,2014-08-14,0.003710,0.001118,0.004256,0.001366,0.004346
5,2014-08-15,-0.003032,-0.003573,0.002679,-0.001478,-0.000061
6,2014-08-18,0.010552,0.005156,0.009716,0.014672,0.008532
7,2014-08-19,0.004801,0.002899,0.004259,0.003513,0.005001
8,2014-08-20,0.003519,0.005559,-0.000227,-0.004267,0.002478
9,2014-08-21,0.003555,0.003759,0.001242,0.002177,0.002950
